# Assignment 6, Qilin Zhou, 02/15/2024

## Question 1

* The dataset is composed of 60,000 32×32-pixel color images (50,000 for training, 10,000 for testing) with 10 classes
Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the ELU activation function. At the output layer use a softmax output layer with 10 neurons


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

2024-02-15 23:53:19.099055: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


###  Load Data

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

### Build DNN

In [13]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
# The dataset is composed of 60,000 32×32-pixel color images 
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for i in range(20):
        model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

# Output layer
model.add(keras.layers.Dense(10, activation="softmax"))

## Question 2

### Using Nadam optimization and early stopping, train the network with Randomized Search for hyperparameter tuning

In [25]:
def build_model(learning_rate=0.001):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
    
    for _ in range(20):
        model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

    model.add(keras.layers.Dense(10, activation="softmax"))

    optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

from scikeras.wrappers import KerasClassifier

keras_classifier = KerasClassifier(build_fn=build_model)

In [27]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = { 
    "model__learning_rate": reciprocal(3e-5, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_classifier, param_distribs, n_iter=10, cv=3, verbose=2)

# Choose 3/4 of the training_full set as training
X_train = X_train_full[37500:]
y_train = y_train_full[37500:]
X_valid = X_train_full[:37500]
y_valid = y_train_full[:37500]

rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/qilinzhou/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
261/261 [==============================] - 13s 20ms/step - loss: 4.4097 - accuracy: 0.1584 - val_loss: 2.2980 - val_accuracy: 0.1915
Epoch 2/100
261/261 [==============================] - 4s 17ms/step - loss: 2.1746 - accuracy: 0.2136 - val_loss: 2.1891 - val_accuracy: 0.2140
Epoch 3/100
261/261 [==============================] - 5s 18ms/step - loss: 2.0815 - accuracy: 0.2347 - val_loss: 2.0390 - val_accuracy: 0.2549
Epoch 4/100
261/261 [==============================] - 5s 18ms/step - loss: 2.0290 - accuracy: 0.2501 - val_loss: 1.9656 - val_accuracy: 0.2787
Epoch 5/100
261/261 [==============================] - 5s 18ms/step - loss: 1.9649 - accuracy: 0.2722 - val_loss: 2.0622 - val_accuracy: 0.2414
Epoch 6/100
261/261 [==============================] - 5s 19ms/step - loss: 1.9550 - accuracy: 0.2869 - val_loss: 2.0110 - val_accuracy: 0.2630
Epoch 7/100
261/261 [==============================] - 5s 20ms/step - loss: 1.9142 - accuracy: 0.2975 - val_loss: 1.9711 - val_accuracy

/Users/qilinzhou/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
261/261 [==============================] - 14s 21ms/step - loss: 6.0584 - accuracy: 0.1236 - val_loss: 2.3185 - val_accuracy: 0.2020
Epoch 2/100
261/261 [==============================] - 5s 19ms/step - loss: 2.2260 - accuracy: 0.1907 - val_loss: 2.1056 - val_accuracy: 0.2223
Epoch 3/100
261/261 [==============================] - 4s 17ms/step - loss: 2.1122 - accuracy: 0.2202 - val_loss: 2.0827 - val_accuracy: 0.2396
Epoch 4/100
261/261 [==============================] - 5s 18ms/step - loss: 2.0722 - accuracy: 0.2426 - val_loss: 2.0735 - val_accuracy: 0.2348
Epoch 5/100
261/261 [==============================] - 5s 18ms/step - loss: 2.0195 - accuracy: 0.2633 - val_loss: 1.9897 - val_accuracy: 0.2728
Epoch 6/100
261/261 [==============================] - 4s 17ms/step - loss: 1.9935 - accuracy: 0.2603 - val_loss: 2.0095 - val_accuracy: 0.2617
Epoch 7/100
261/261 [==============================] - 4s 17ms/step - loss: 1.9540 - accuracy: 0.2797 - val_loss: 1.9693 - val_accuracy

...

/Users/qilinzhou/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
261/261 [==============================] - 21s 31ms/step - loss: 11.3332 - accuracy: 0.1224 - val_loss: 2.7420 - val_accuracy: 0.1481
Epoch 2/100
261/261 [==============================] - 7s 26ms/step - loss: 2.4605 - accuracy: 0.1673 - val_loss: 2.3159 - val_accuracy: 0.1782
Epoch 3/100
261/261 [==============================] - 7s 26ms/step - loss: 2.2259 - accuracy: 0.1976 - val_loss: 2.2101 - val_accuracy: 0.2030
Epoch 4/100
261/261 [==============================] - 7s 26ms/step - loss: 2.1259 - accuracy: 0.2208 - val_loss: 2.0688 - val_accuracy: 0.2301
Epoch 5/100
261/261 [==============================] - 7s 26ms/step - loss: 2.0691 - accuracy: 0.2412 - val_loss: 2.1047 - val_accuracy: 0.2216
Epoch 6/100
261/261 [==============================] - 7s 26ms/step - loss: 2.0327 - accuracy: 0.2510 - val_loss: 2.0326 - val_accuracy: 0.2523
Epoch 7/100
261/261 [==============================] - 7s 26ms/step - loss: 1.9846 - accuracy: 0.2714 - val_loss: 2.0323 - val_accurac

/Users/qilinzhou/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
391/391 [==============================] - 24s 27ms/step - loss: 5.4214 - accuracy: 0.1706 - val_loss: 2.1093 - val_accuracy: 0.2274
Epoch 2/100
391/391 [==============================] - 9s 24ms/step - loss: 2.0906 - accuracy: 0.2312 - val_loss: 2.0447 - val_accuracy: 0.2350
Epoch 3/100
391/391 [==============================] - 9s 23ms/step - loss: 2.0267 - accuracy: 0.2501 - val_loss: 2.0111 - val_accuracy: 0.2455
Epoch 4/100
391/391 [==============================] - 9s 23ms/step - loss: 1.9834 - accuracy: 0.2686 - val_loss: 1.9709 - val_accuracy: 0.2808
Epoch 5/100
391/391 [==============================] - 10s 25ms/step - loss: 1.9232 - accuracy: 0.2940 - val_loss: 1.8778 - val_accuracy: 0.3170
Epoch 6/100
391/391 [==============================] - 10s 27ms/step - loss: 1.8959 - accuracy: 0.3007 - val_loss: 1.8924 - val_accuracy: 0.3001
Epoch 7/100
391/391 [==============================] - 10s 27ms/step - loss: 1.8629 - accuracy: 0.3181 - val_loss: 1.8650 - val_accur

RandomizedSearchCV(cv=3,
                   estimator=KerasClassifier(build_fn=<function build_model at 0x15b7a5080>),
                   param_distributions={'model__learning_rate': [0.01192544089977353,
                                                                 0.001095892084442962,
                                                                 0.00305434887884912,
                                                                 0.008359247899717575,
                                                                 0.00018161291448501717,
                                                                 0.0002332428167492523,
                                                                 3.0130324195205462e-05,
                                                                 0.023248966270150648,
                                                                 0.016972895545961188,
                                                                 0.0058996472607239...
                                                                 0.0021167528362478283,
                                                                 0.022888087546062717,
                                                                 0.015336467019355123,
                                                                 0.0036930543533677876,
                                                                 5.848479974701813e-05,
                                                                 0.00019934868157775343,
                                                                 0.0002737104857384861,
                                                                 0.005842132841357289,
                                                                 0.007653653423987863,
                                                                 0.0004289825472562583,
                                                                 0.0005379888124804675,
                                                                 4.079889234703265e-05,
                                                                 0.0002210708138087823,
                                                                 0.0006776052875363995,
                                                                 6.730857464902144e-05,
                                                                 0.0006869476417826836, ...]},
                   verbose=2)

In [29]:
rnd_search_cv.best_params_

{'model__learning_rate': 0.00042199648026968004}

* The best tuned learning rate is thus set to be 0.000422

In [34]:
model = build_model(learning_rate=0.000422)

In [37]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 279.0726 - accuracy: 0.1097


[279.0726013183594, 0.10970000177621841]

* The current model accuracy is around 10.97%.

## Question 3

### Add Batch Normalization and compare the learning curves

In [30]:
import os
root_logdir = os.path.join(os.curdir, "my_cifar10_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [38]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=0.000422) 
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5",
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_test, y_test)

Epoch 1/100
388/391 [============================>.] - ETA: 0s - loss: 2.0347 - accuracy: 0.2724

/Users/qilinzhou/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


391/391 [==============================] - 30s 27ms/step - loss: 2.0346 - accuracy: 0.2726 - val_loss: 1.8385 - val_accuracy: 0.3398
Epoch 2/100
391/391 [==============================] - 9s 22ms/step - loss: 1.8096 - accuracy: 0.3479 - val_loss: 1.7461 - val_accuracy: 0.3697
Epoch 3/100
391/391 [==============================] - 9s 24ms/step - loss: 1.7163 - accuracy: 0.3832 - val_loss: 1.7019 - val_accuracy: 0.3878
Epoch 4/100
391/391 [==============================] - 10s 25ms/step - loss: 1.6570 - accuracy: 0.4109 - val_loss: 1.6471 - val_accuracy: 0.4141
Epoch 5/100
391/391 [==============================] - 10s 26ms/step - loss: 1.6091 - accuracy: 0.4320 - val_loss: 1.6303 - val_accuracy: 0.4215
Epoch 6/100
391/391 [==============================] - 9s 22ms/step - loss: 1.5664 - accuracy: 0.4429 - val_loss: 1.6357 - val_accuracy: 0.4199
Epoch 7/100
391/391 [==============================] - 9s 22ms/step - loss: 1.5365 - accuracy: 0.4590 - val_loss: 1.6212 - val_accuracy: 0.4234
E

[1.5583374500274658, 0.4542999863624573]

In [47]:
%reload_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs --port=6006 --host=0.0.0.0

Reusing TensorBoard on port 6006 (pid 2213), started 0:01:33 ago. (Use '!kill 2213' to kill it.)

* Batch Normalization makes the model converge faster, taking only 20 epochs to reach the lowest validation loss, with test accuracy around 45.43%. This is a significant improvement from the prior model with only 10.97% accuracy. The training speed is not faster, with around 9s for each epoch, while basline model has around 7s for each epoch, on average.

## Question 4

###  Replacing Batch Normalization with SELU

* standardize the input features

In [48]:
means = X_train.mean(axis=0)
stds = X_train.std(axis=0)
X_train_scaled = (X_train - means) / stds
X_valid_scaled = (X_valid - means) / stds
X_test_scaled = (X_test - means) / stds

* use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers

In [52]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=0.000422) 
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [53]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5",
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_test_scaled, y_test)

Epoch 1/100
391/391 [==============================] - 19s 24ms/step - loss: 2.0484 - accuracy: 0.2646 - val_loss: 1.8849 - val_accuracy: 0.3237
Epoch 2/100
391/391 [==============================] - 6s 16ms/step - loss: 1.8558 - accuracy: 0.3382 - val_loss: 1.8375 - val_accuracy: 0.3391
Epoch 3/100
391/391 [==============================] - 6s 16ms/step - loss: 1.7538 - accuracy: 0.3695 - val_loss: 1.7595 - val_accuracy: 0.3669
Epoch 4/100
391/391 [==============================] - 6s 16ms/step - loss: 1.6596 - accuracy: 0.4082 - val_loss: 1.7577 - val_accuracy: 0.3809
Epoch 5/100
391/391 [==============================] - 6s 16ms/step - loss: 1.6027 - accuracy: 0.4314 - val_loss: 1.7202 - val_accuracy: 0.3973
Epoch 6/100
391/391 [==============================] - 6s 16ms/step - loss: 1.5226 - accuracy: 0.4626 - val_loss: 1.7082 - val_accuracy: 0.4054
Epoch 7/100
391/391 [==============================] - 6s 16ms/step - loss: 1.4634 - accuracy: 0.4854 - val_loss: 1.6946 - val_accuracy

[1.6753941774368286, 0.4250999987125397]

In [54]:
%tensorboard --logdir=./my_cifar10_logs --port=6006 --host=0.0.0.0

Reusing TensorBoard on port 6006 (pid 2213), started 0:18:19 ago. (Use '!kill 2213' to kill it.)

* SELU method converges faster, taking only 17 epochs to reach the lowest validation loss. However, it does not improve the test accuracy from Batch Normalization, with 42.51% test accuracy.

## Question 5

### Try regularizing the model with alpha dropout. 

In [57]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.AlphaDropout(rate=0.2))
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=0.000422) 
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [59]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_model.h5",
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20,
                                                  restore_best_weights=True)

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

model = keras.models.load_model("my_cifar10_alpha_model.h5")
model.evaluate(X_test_scaled, y_test)

Epoch 1/100
391/391 [==============================] - 17s 28ms/step - loss: 2.2136 - accuracy: 0.1725 - val_loss: 3.4971 - val_accuracy: 0.1512
Epoch 2/100
391/391 [==============================] - 11s 28ms/step - loss: 2.1885 - accuracy: 0.1643 - val_loss: 5.6947 - val_accuracy: 0.1810
Epoch 3/100
391/391 [==============================] - 10s 26ms/step - loss: 2.1484 - accuracy: 0.1589 - val_loss: 20.0929 - val_accuracy: 0.1577
Epoch 4/100
391/391 [==============================] - 10s 25ms/step - loss: 2.1233 - accuracy: 0.1669 - val_loss: 16.0944 - val_accuracy: 0.1642
Epoch 5/100
391/391 [==============================] - 11s 28ms/step - loss: 2.0941 - accuracy: 0.1747 - val_loss: 25.3057 - val_accuracy: 0.1675
Epoch 6/100
391/391 [==============================] - 11s 28ms/step - loss: 2.0868 - accuracy: 0.1749 - val_loss: 32.7706 - val_accuracy: 0.1279
Epoch 7/100
391/391 [==============================] - 11s 28ms/step - loss: 2.0696 - accuracy: 0.1822 - val_loss: 23.0864 - v

[3.5354628562927246, 0.15150000154972076]

In [60]:
%tensorboard --logdir=./my_cifar10_logs --port=6006 --host=0.0.0.0

Reusing TensorBoard on port 6006 (pid 2213), started 0:35:09 ago. (Use '!kill 2213' to kill it.)

The model with alpha dropout increases the baseline model test accuracy from 10.97% to 15.15%.

### using MC Dropout

In [61]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [63]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [64]:
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_pred = np.argmax(y_proba, axis=1)

In [65]:
accuracy = np.mean(y_pred == y_test[:, 0])
accuracy

0.1672

* We achieved higher acccuracy from MC Dropout, which is now 16.72%,  compared with Alpha Dropout (15.15%).

## Question 6

### 1cycle scheduling

In [66]:
import math
K = keras.backend

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [70]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.2))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(learning_rate=0.000422)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [73]:
batch_size = 128
n_epochs = 25
onecycle = OneCycleScheduler(math.ceil(len(X_train_scaled) / batch_size) * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[onecycle])

Epoch 1/25
98/98 [==============================] - 5s 48ms/step - loss: 0.4806 - accuracy: 0.8316 - val_loss: 3.0436 - val_accuracy: 0.4378
Epoch 2/25
98/98 [==============================] - 3s 26ms/step - loss: 0.5122 - accuracy: 0.8183 - val_loss: 3.1088 - val_accuracy: 0.4355
Epoch 3/25
98/98 [==============================] - 3s 27ms/step - loss: 0.5980 - accuracy: 0.7923 - val_loss: 2.9893 - val_accuracy: 0.4325
Epoch 4/25
98/98 [==============================] - 3s 26ms/step - loss: 0.6939 - accuracy: 0.7579 - val_loss: 2.8764 - val_accuracy: 0.4311
Epoch 5/25
98/98 [==============================] - 3s 26ms/step - loss: 0.7290 - accuracy: 0.7410 - val_loss: 2.8321 - val_accuracy: 0.4160
Epoch 6/25
98/98 [==============================] - 3s 27ms/step - loss: 0.8117 - accuracy: 0.7159 - val_loss: 2.7948 - val_accuracy: 0.4219
Epoch 7/25
98/98 [==============================] - 3s 30ms/step - loss: 0.8264 - accuracy: 0.7078 - val_loss: 2.5387 - val_accuracy: 0.4100
Epoch 8/25
98

In [74]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 4.6512 - accuracy: 0.4491


[4.65122127532959, 0.44909998774528503]

* The training epoch is much faster, each taking only around 4 seconds. The test accuracy is around 45%, which is slightly slower than Batch Normalization.